In [1]:
import os
import time
import duckdb
import pandas as pd
from dotenv import load_dotenv

In [12]:
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [31]:
# to receive feedback from SQL queries
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = None  # None means unlimited
%config SqlMagic.style = "SINGLE_BORDER"  

# some people like to use this
# %config SqlMagic.autopandas = True
# but this will make the notebook slow

displaylimit: Value None will be treated as 0 (no limit)

In [33]:
%%sql foo <<
select 42 as my_answer

In [34]:
print(foo)

┌───────────┐
│ my_answer │
├───────────┤
│     42    │
└───────────┘


In [40]:
%%sql column_info <<
describe
select * from read_json_auto('./data/goodreads_books.json.gz')

In [50]:
print(column_info)

┌──────────────────────┬────────────────────────────────────────────┬──────┬──────┬─────────┬───────┐
│     column_name      │                column_type                 │ null │ key  │ default │ extra │
├──────────────────────┼────────────────────────────────────────────┼──────┼──────┼─────────┼───────┤
│         isbn         │                  VARCHAR                   │ YES  │ None │   None  │  None │
│  text_reviews_count  │                  VARCHAR                   │ YES  │ None │   None  │  None │
│        series        │                  BIGINT[]                  │ YES  │ None │   None  │  None │
│     country_code     │                  VARCHAR                   │ YES  │ None │   None  │  None │
│    language_code     │                  VARCHAR                   │ YES  │ None │   None  │  None │
│   popular_shelves    │   STRUCT(count BIGINT, "name" VARCHAR)[]   │ YES  │ None │   None  │  None │
│         asin         │                  VARCHAR                   │ YES  │ None 

In [42]:
%%sql book_sample <<
select isbn text_reviews_count, country_code, average_rating,
language_code, url, book_id, title
from read_json_auto('./data/goodreads_books.json.gz')
using sample 5;

In [43]:
print(book_sample)

┌────────────────────┬──────────────┬────────────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┬───────────────────────────────────────────┐
│ text_reviews_count │ country_code │ average_rating │ language_code │                                                                   url                                                                   │ book_id  │                   title                   │
├────────────────────┼──────────────┼────────────────┼───────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────┼───────────────────────────────────────────┤
│                    │      US      │      3.88      │               │      <a href=https://www.goodreads.com/book/show/25250702-alexia-eden>https://www.goodreads.com/book/show/25250702-alexia-eden</a>      │